# ⏩ Speculative Decoding: Jak przyspieszyć LLM 2x bez utraty jakości?

Wielkie modele (Target Model) są ograniczone przez **Memory Bandwidth** (przepustowość pamięci). Załadowanie wag trwa dłużej niż same obliczenia.
Dlatego uruchomienie modelu dla 1 tokena kosztuje tyle samo czasu, co dla 5 tokenów (jeśli robimy to równolegle).

**Algorytm Spekulacyjny:**
1.  **Draft:** Mały model (np. GPT-2) generuje szybko `K` tokenów (np. "Ala ma kota").
2.  **Verify:** Duży model (np. GPT-4) dostaje ten ciąg i liczy prawdopodobieństwa dla wszystkich pozycji naraz.
3.  **Accept/Reject:** Sprawdzamy, czy Duży Model zgadza się z Małym.
    *   Akceptujemy prefiks, który jest zgodny.
    *   Resztę odrzucamy i generujemy poprawny token z Dużego Modelu.

Zrobimy symulację tego procesu, używając sztucznych opóźnień (`time.sleep`), żeby pokazać zysk czasowy.

In [1]:
import time
import random
import torch
import numpy as np

# Konfiguracja symulacji
# Target Model jest mądry (dokładny), ale wolny
TARGET_TIME_PER_TOKEN = 0.1  # 100 ms
# Draft Model jest głupi (często się myli), ale szybki
DRAFT_TIME_PER_TOKEN = 0.01  # 10 ms

# Długość spekulacji (ile tokenów zgadujemy naraz?)
K_LOOKAHEAD = 4 

# Nasze "Słownictwo" (uproszczone)
target_sentence = "Sztuczna inteligencja zmienia świat na lepsze".split()
vocab = {word: i for i, word in enumerate(target_sentence)}
vocab_inv = {i: word for word, i in vocab.items()}

print(f"Zdanie docelowe: {target_sentence}")
print(f"Draft jest {TARGET_TIME_PER_TOKEN / DRAFT_TIME_PER_TOKEN:.0f}x szybszy od Targetu.")

Zdanie docelowe: ['Sztuczna', 'inteligencja', 'zmienia', 'świat', 'na', 'lepsze']
Draft jest 10x szybszy od Targetu.


## Symulacja Modeli

Stworzymy dwie funkcje:
1.  `target_model`: Zawsze zwraca poprawne słowo, ale "myśli" długo.
2.  `draft_model`: Myśli szybko, ale ma np. 70% szans na trafienie (symulacja mniejszej inteligencji).

In [2]:
class MockModel:
    def __init__(self, name, latency, accuracy, true_sentence):
        self.name = name
        self.latency = latency
        self.accuracy = accuracy
        self.true_sentence = true_sentence

    def forward(self, input_ids):
        """
        Zwraca następny token (lub listę tokenów).
        """
        # Symulacja czasu obliczeń
        time.sleep(self.latency)
        
        current_len = len(input_ids)
        if current_len >= len(self.true_sentence):
            return "<EOS>" # Koniec zdania
        
        # Prawdziwy token, który powinien być
        true_token = self.true_sentence[current_len]
        
        # Czy model zgadł?
        if random.random() < self.accuracy:
            return true_token
        else:
            # Pomyłka (losowe słowo ze słownika)
            return random.choice(list(vocab.keys()))

# Inicjalizacja
# Target: 100% dokładności (to nasza wyrocznia), wolny
target_model = MockModel("Target (Big)", TARGET_TIME_PER_TOKEN, 1.0, target_sentence)

# Draft: 70% dokładności, szybki
draft_model = MockModel("Draft (Small)", DRAFT_TIME_PER_TOKEN, 0.7, target_sentence)

print("Modele gotowe.")

Modele gotowe.


## Metoda 1: Standardowa Generacja (Autoregresyjna)

To jest sposób, w jaki normalnie działa ChatGPT. Jeden token po drugim, używając tylko dużego modelu.

In [3]:
def standard_generation():
    current_text = []
    start_time = time.time()
    
    print("--- STANDARD GENERATION ---")
    while len(current_text) < len(target_sentence):
        # Generujemy 1 token Dużym Modelem
        token = target_model.forward(current_text)
        current_text.append(token)
        print(f"Generated: {token}")
        
    total_time = time.time() - start_time
    return total_time, len(current_text)

time_std, tokens_std = standard_generation()
print(f"\nCzas Standardowy: {time_std:.4f} s")
print(f"Prędkość: {tokens_std / time_std:.2f} tokens/s")

--- STANDARD GENERATION ---
Generated: Sztuczna
Generated: inteligencja
Generated: zmienia
Generated: świat
Generated: na
Generated: lepsze

Czas Standardowy: 0.6032 s
Prędkość: 9.95 tokens/s


## Metoda 2: Speculative Decoding

Teraz algorytm spekulacyjny:
1.  Pętla główna.
2.  **Krok Draftu:** Mały model generuje `K` tokenów jeden po drugim (szybko).
3.  **Krok Weryfikacji:** Duży model sprawdza te `K` tokenów + 1 dodatkowy (równolegle).
    *   *Uwaga: W naszej symulacji Pythona "równoległość" symulujemy tym, że `target_model` sprawdza listę tokenów w tym samym czasie co pojedynczy token (bo koszt to głównie załadowanie wag).*
4.  **Akceptacja:** Bierzemy tyle tokenów, ile się zgadza.

In [4]:
def speculative_generation():
    current_text = []
    start_time = time.time()
    
    print("--- SPECULATIVE GENERATION ---")
    
    while len(current_text) < len(target_sentence):
        # 1. DRAFT PHASE (Mały model zgaduje K razy)
        draft_tokens = []
        temp_context = current_text.copy()
        
        for _ in range(K_LOOKAHEAD):
            if len(temp_context) >= len(target_sentence): break
            token = draft_model.forward(temp_context)
            draft_tokens.append(token)
            temp_context.append(token)
            
        # 2. VERIFICATION PHASE (Duży model sprawdza)
        # W prawdziwym GPU to dzieje się w JEDNYM przebiegu (batch)
        # Symulujemy koszt jednego uruchomienia dużego modelu
        time.sleep(TARGET_TIME_PER_TOKEN) 
        
        # Sprawdzamy poprawność (Verify)
        # Target model generuje "swoją wersję" dla każdego kroku w drafcie
        # (W prawdziwym świecie liczy prawdopodobieństwa P(x))
        accepted_tokens = []
        for i, draft_tok in enumerate(draft_tokens):
            # Jaki token powinien być na tej pozycji wg Targetu?
            true_pos = len(current_text) + i
            if true_pos >= len(target_sentence): break
            
            target_tok = target_sentence[true_pos]
            
            if draft_tok == target_tok:
                accepted_tokens.append(draft_tok)
            else:
                # Błąd! Odrzucamy resztę draftu
                # Target model "poprawia" ten jeden błąd
                accepted_tokens.append(target_tok)
                break # Przerywamy akceptację
        
        # Aktualizacja tekstu
        current_text.extend(accepted_tokens)
        
        # Logowanie
        status = "✅" if len(accepted_tokens) > 1 else "❌"
        print(f"Draft: {draft_tokens} -> Zaakceptowano: {accepted_tokens} {status}")

    total_time = time.time() - start_time
    return total_time, len(current_text)

time_spec, tokens_spec = speculative_generation()
print(f"\nCzas Spekulacyjny: {time_spec:.4f} s")
print(f"Prędkość: {tokens_spec / time_spec:.2f} tokens/s")

--- SPECULATIVE GENERATION ---
Draft: ['Sztuczna', 'inteligencja', 'świat', 'świat'] -> Zaakceptowano: ['Sztuczna', 'inteligencja', 'zmienia'] ✅
Draft: ['świat', 'na', 'lepsze'] -> Zaakceptowano: ['świat', 'na', 'lepsze'] ✅

Czas Spekulacyjny: 0.2736 s
Prędkość: 21.93 tokens/s


In [5]:
# PODSUMOWANIE
speedup = time_std / time_spec
print("-" * 30)
print(f"Standard: {time_std:.4f}s")
print(f"Speculative: {time_spec:.4f}s")
print(f"🚀 PRZYSPIESZENIE: {speedup:.2f}x")

if speedup > 1.0:
    print("Sukces! Mały model skutecznie pomógł dużemu.")
else:
    print("Porażka. Mały model mylił się zbyt często (narzut czasowy draftu zjadł zysk).")

------------------------------
Standard: 0.6032s
Speculative: 0.2736s
🚀 PRZYSPIESZENIE: 2.20x
Sukces! Mały model skutecznie pomógł dużemu.


## 🧠 Podsumowanie: Hazard, który się opłaca

Jeśli mały model ma skuteczność (Acceptance Rate) powyżej ~50-60%, spekulacja się opłaca.
Jeśli mały model jest beznadziejny, spekulacja tylko spowalnia (bo tracimy czas na draft, który i tak idzie do kosza).

**Dlaczego to działa na GPU?**
Uruchomienie modelu GPT-4 dla sekwencji o długości 1 (input) kosztuje np. 50ms.
Uruchomienie go dla sekwencji o długości 5 (input) też kosztuje ok. 50-55ms.
Dzięki temu **weryfikacja 5 tokenów jest prawie darmowa** w porównaniu do generowania ich pojedynczo.

To obecnie standardowa technika w bibliotekach takich jak **vLLM** czy **HuggingFace TGI**.